In [2]:
import pandas as pd
import os
import spacy
from spacy.pipeline import EntityRuler
from spacy import displacy
import seaborn as sns
import matplotlib.pyplot as plt
import jsonlines
import textract
import PyPDF2
from spacy.language import Language
from pdfminer.high_level import extract_text
import streamlit as st

In [3]:
PROJECT_DIR = os.path.dirname(os.getcwd()) + '/mesure/'

skill_pattern_path = PROJECT_DIR + "data/skill_patterns.jsonl"
file = PROJECT_DIR + '/CV/Philippe_Nguyen_CV_08_21.pdf'
nlp = spacy.load('en_core_web_sm')
pdfreader = "textract"

In [4]:
def extract_text_from_pdf(file, pdfreader):
    '''Opens and reads in a PDF file from path'''
    
    fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
    page_count = fileReader.getNumPages()
    # text = [fileReader.getPage(i).extractText() for i in range(page_count)]
    if pdfreader == "textract":
        text = str(textract.process(file)).replace("\\n", " ")
    elif pdfreader == "pdfminer":
        text = extract_text(file).replace("\n", " ")
    return text

In [5]:
with jsonlines.open(PROJECT_DIR + "data/skill_patterns.jsonl") as f:
    created_entities = [line['label'].upper() for line in f.iter()]

    
def add_newruler_to_pipeline(skill_pattern_path):
    '''Reads in all created patterns from a JSONL file and adds it to the pipeline after PARSER and before NER'''
        
    ruler = nlp.add_pipe("entity_ruler", after='parser')
    ruler.from_disk(skill_pattern_path)
    

def visualize_entity_ruler(entity_list, doc):
    '''Visualize the Skill entities of a doc'''
    
    options = {"ents": entity_list}
    displacy.render(doc, style='ent', options=options)
    
add_newruler_to_pipeline(skill_pattern_path)

# skillset_dict = create_skillset_dict(resume_names, resume_texts)

In [6]:
def create_skill_set(doc):
    '''Create a set of the extracted skill entities of a doc'''
    
    return set([ent.label_.upper().split('|')[1] for ent in doc.ents if 'SKILL' in ent.label_])

In [14]:
text = extract_text_from_pdf(file, pdfreader)
doc = nlp(text)
visualize_entity_ruler(created_entities, doc)

In [15]:
create_skill_set(doc)

{'ANDROID',
 'C',
 'COMPUTER-SCIENCE',
 'COMPUTER-VISION',
 'CSS',
 'DART',
 'DATA-ANALYSIS',
 'DATA-MINING',
 'DATABASES',
 'DEBIAN',
 'DEEP-LEARNING',
 'DEPLOYMENT',
 'DESIGN',
 'DJANGO',
 'FIGMA',
 'FINANCE',
 'FLASK',
 'FLUTTER',
 'FRAMEWORK',
 'GITHUB',
 'HTML',
 'IMAGE-PROCESSING',
 'INFERENCE',
 'IOS',
 'JAVA',
 'KOTLIN',
 'LANGUAGES',
 'LIBRARY',
 'LINEAR-PROGRAMMING',
 'LINUX',
 'MACHINE-LEARNING',
 'MACOS',
 'MARKETING',
 'MOBILE',
 'NUMPY',
 'OBJECT-DETECTION',
 'PANDAS',
 'PYTHON',
 'REINFORCEMENT-LEARNING',
 'SCALA',
 'SOFTWARE-ENGINEERING',
 'TEXT-PROCESSING',
 'TIME-SERIES',
 'WINDOWS'}

In [16]:
number = st.slider("Pick a number", 0, 100)

In [17]:
vacancy = """
Vous êtes titulaire d’un Master ou doctorat en science, science informatique, mathématiques, statistiques ou dans un domaine équivalent.
Vous avez 3 à 5 ans d’expérience réussie et pertinente dans un poste similaire.
Vous maîtrisez le « scripting » et le « programming » ainsi que les logiciels tels que Python et R.
Vous avez une expérience avérée avec les environnements Cloud, idéalement « Azure Suite » : Azure synapse, Azure ML, Azure DevOps, Azure Kubernetes Services.
Vous avez de l’expérience dans le développement des logiciels (Agile, Version Control, Git. Une expérience avec MLOps CI/CD est un atout.
Vous avez de l’expérience dans le domaine de l’éducation et/ou du e-commerce.
Vous êtes dotés d’excellentes compétences en communication et avez une capacité à comprendre et résoudre des problèmes complexes.
"""
                                                               
doc2 = nlp(vacancy)
visualize_entity_ruler(created_entities, doc2)

In [18]:
create_skill_set(doc2)

{'AGILE',
 'AZURE',
 'COMMERCE',
 'DEVOPS',
 'GIT',
 'KUBERNETES',
 'MACHINE-LEARNING',
 'PYTHON'}

In [20]:
create_skill_set(doc).intersection(create_skill_set(doc2))

{'MACHINE-LEARNING', 'PYTHON'}